In [ ]:
pip install efficientnet_pytorch

install

In [ ]:
# train.py
import os, cv2, torch, pandas as pd, numpy as np
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch.optim as optim

# Configs
DATA_DIR = "/kaggle/input/soil-classification-1/soil_classification-2025"
TRAIN_IMG = os.path.join(DATA_DIR, "train")
TRAIN_CSV = os.path.join(DATA_DIR, "train_labels.csv")
OUTPUT_DIR = "/kaggle/working"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N_FOLDS, BATCH_SIZE, NUM_EPOCHS, NUM_CLASSES = 5, 32, 20, 4
LR, WD = 3e-4, 1e-5

label2idx = {'Alluvial soil': 0, 'Black Soil': 1, 'Clay soil': 2, 'Red soil': 3}
df = pd.read_csv(TRAIN_CSV)
df['soil_type_idx'] = df['soil_type'].map(label2idx)

# Transforms
transform_train = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_val = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def load_clahe_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (300, 300))
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = cv2.createCLAHE(2.0, (8,8)).apply(l)
    return Image.fromarray(cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2RGB))

class SoilDataset(Dataset):
    def __init__(self, df, img_dir, transform):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = load_clahe_image(os.path.join(self.img_dir, row['image_id']))
        return self.transform(img), row['soil_type_idx']

def train_one_fold(train_idx, val_idx, fold):
    train_df, val_df = df.loc[train_idx], df.loc[val_idx]
    weights = 1.0 / train_df['soil_type_idx'].value_counts().sort_index().values
    sample_weights = train_df['soil_type_idx'].map(lambda x: weights[x]).values
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

    train_loader = DataLoader(SoilDataset(train_df, TRAIN_IMG, transform_train), batch_size=BATCH_SIZE, sampler=sampler, num_workers=2)
    val_loader = DataLoader(SoilDataset(val_df, TRAIN_IMG, transform_val), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    model = EfficientNet.from_pretrained('efficientnet-b3')
    model._fc = nn.Linear(model._fc.in_features, NUM_CLASSES)
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

    best_min_f1 = 0.0
    for epoch in range(1, NUM_EPOCHS + 1):
        model.train()
        train_losses = []
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(imgs), labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        scheduler.step()

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                preds = model(imgs).argmax(1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        min_f1 = f1_score(all_labels, all_preds, average=None).min()
        print(f"Fold {fold} Epoch {epoch}: Loss {np.mean(train_losses):.4f}, Min F1 {min_f1:.4f}")
        if min_f1 > best_min_f1:
            best_min_f1 = min_f1
            torch.save(model.state_dict(), f"{OUTPUT_DIR}/model_fold{fold}.pth")

# K-Fold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['soil_type_idx'])):
    print(f"\n=== Fold {fold} ===")
    train_one_fold(train_idx, val_idx, fold)


Training

In [ ]:
# inference.py
import os, cv2, torch, pandas as pd, numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

# Configs
DATA_DIR = "/kaggle/input/soil-classification-1/soil_classification-2025"
TEST_IMG = os.path.join(DATA_DIR, "test")
TEST_IDS = os.path.join(DATA_DIR, "test_ids.csv")
OUTPUT_DIR = "/kaggle/working"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE, NUM_CLASSES, N_FOLDS = 32, 4, 5
idx2label = {0: 'Alluvial soil', 1: 'Black Soil', 2: 'Clay soil', 3: 'Red soil'}

def load_clahe_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (300, 300))
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = cv2.createCLAHE(2.0, (8,8)).apply(l)
    return Image.fromarray(cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2RGB))

transform_val = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class TestDataset(Dataset):
    def __init__(self, image_ids, img_dir, transform):
        self.image_ids, self.img_dir, self.transform = image_ids, img_dir, transform
    def __len__(self): return len(self.image_ids)
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img = load_clahe_image(os.path.join(self.img_dir, img_id))
        return self.transform(img), img_id

test_ids = pd.read_csv(TEST_IDS)
test_list = test_ids['image_id'].tolist()
test_loader = DataLoader(TestDataset(test_list, TEST_IMG, transform_val), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

ensemble_probs = {img_id: [] for img_id in test_list}
for fold in range(N_FOLDS):
    model = EfficientNet.from_name('efficientnet-b3')
    model._fc = torch.nn.Linear(model._fc.in_features, NUM_CLASSES)
    model.load_state_dict(torch.load(f"{OUTPUT_DIR}/model_fold{fold}.pth"))
    model = model.to(DEVICE).eval()

    with torch.no_grad():
        for imgs, img_ids in test_loader:
            imgs = imgs.to(DEVICE)
            probs = model(imgs).cpu().softmax(1).numpy()
            for i in range(len(img_ids)):
                ensemble_probs[img_ids[i]].append(probs[i])

# Write submission
out = []
for img_id in test_list:
    avg = np.mean(ensemble_probs[img_id], axis=0)
    out.append({'image_id': img_id, 'soil_type': idx2label[int(avg.argmax())]})

pd.DataFrame(out).to_csv(f"{OUTPUT_DIR}/submission.csv", index=False)
print("submission.csv written")


inference